# Lesson 13

Much of the begninning of the video is a review of the basics of the neural network (as matrix multiplications and nonlinear pointwise function).  In the beginning he uses a simple 2 layer network with 50 hidden layers and 1 output layer that predicts the digit.  The loss is then the MSE of the difference between the actual digit and the output of this last layer. This is a simplified model that is not correct, but it is a good starting point to understand the basics of the neural network. 

ADD: Jacobian

In [ ]:
## TODO import libraries, and set up the problem as in the course repo.

He then discusses that with the loss in hand, we need gradients to know how to adjust the weights. There is a long discussion giving some intuition of the chain rule, which I will not repeat here, as I am quite familier with calculus and the chain rule.
The chain rule is important because we have a function (the prediction) that is a composition of other functions.
 

The approach is to start at the end, take the derivative, and keep multiplying (chain rule!) by the derivative of the next function until we reach the beginning. This is the backpropagation algorithm in essence.

He walks through this in a messy way at about 48 minutes in the video.  He uses the built in debugger to understand it.